In [1]:
import h5py
import glob
import os
import sys
from pathlib import Path
import torch
import pandas as pd
import numpy as np
import logging
from deeprankcore.Trainer import Trainer
from deeprankcore.ginet import GINet
from deeprankcore.DataSet import HDF5DataSet, save_hdf5_keys
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.metrics import (
    roc_curve,
    precision_recall_curve,
    auc,
    average_precision_score,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
    matthews_corrcoef)

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#################### To fill
exp_id = 'exp3'
####################
exp_df = pd.read_excel('experiments_log.xlsx', index_col='exp_id')
exp_df

,input_data_path,protein_class,target_data,res_data,task,node_features,edge_features,net,batch_size,optimizer,...,test_accuracy,train_mcc,valid_mcc,test_mcc,train_f1,valid_f1,test_f1,train_rmse,valid_rmse,test_rmse
exp_id,,,,,,,,,,,,,,,,,,,,,
exp3,/Users/giuliacrocioni/Desktop/docs/eScience/pr...,I,BA,residue,classif,"['res_type', 'res_charge', 'res_size', 'polari...","['same_chain', 'distance', 'covalent', 'electr...",<class 'deeprankcore.naive_gnn.NaiveNetwork'>,16,<class 'torch.optim.adam.Adam'>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
epoch = exp_df.loc[exp_id].epoch
exp_path = os.path.join('experiments', exp_id)
metrics_path = os.path.join(exp_path, 'metrics')
img_path = os.path.join(exp_path, 'images')

df_summ = pd.read_hdf(os.path.join(metrics_path, 'summary_data.hdf5'), 'summary')
metrics = pd.read_hdf(os.path.join(metrics_path, 'metrics.hdf5'), 'metrics')
print(df_summ.shape)
print(metrics.shape)

df = pd.merge(metrics, df_summ[['cluster', 'entry']], on="entry", how="outer")
df.sort_values(by=['epoch'], inplace = True)

print(df.shape)

print(df.cluster.notna().all())

(50, 4)
(510, 6)
(510, 7)
True


## Loss vs epochs

In [5]:
fig = px.line(
    df[(df.phase =='training') | (df.phase =='validation')],
    x='epoch',
    y='loss',
    color='phase',
    markers=True)

fig.update_layout(
    xaxis_title='Epoch #',
    yaxis_title='Loss',
    width=800, height=500,
    title='Loss vs epochs',
    title_x=0.5,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4),
    legend=dict(
        yanchor="top",
        #y=0.99
        xanchor="right",
        x=0.99
        )
)

fig.add_vline(x=epoch, line_width=3, line_dash="dash", line_color="green")
fig.show()
fig.write_html(os.path.join(img_path, 'loss_epoch.html'))

## Binary classification metrics (for best/saved epoch)

In [6]:
df_plot = df[(df.epoch == epoch) | ((df.epoch == 0) & (df.phase == 'testing'))]
print(df_plot.shape)
y_true = df_plot.target
y_score = np.array(df_plot.output.values.tolist())[:, 1]

(50, 7)


### Only target distributions per target and cluster (data exploration)

In [7]:
clusters = df_plot.cluster.unique()
clusters.sort()
clusters = list(clusters)
df_plot['target_str'] = df_plot['target'].astype(str)
# The histogram of scores compared to true labels
fig = px.histogram(
    df_plot,
    x='target_str',
    color=df_plot.cluster,
    facet_row='phase',
    category_orders={'phase': [
        'training',
        "validation",
        "testing"],
        'cluster': clusters}
)
fig.update_layout(
    width=600, height=600,
    showlegend=True,
    title='Target',
    title_x=0.5,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    )
)
fig.update_yaxes(range=[0, 2700], constrain='domain')
fig.update_layout(bargap=0.30,bargroupgap=0.0)
fig.show()
fig.write_html(os.path.join(img_path, 'target_only.html'))

/var/folders/js/v54t24f101585t5fkkx3qpkh0000gn/T/ipykernel_19368/1327335858.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Target and score distributions per target and cluster

In [8]:
clusters = df_plot.cluster.unique()
clusters.sort()
clusters = list(clusters)
# The histogram of scores compared to true labels
fig = px.histogram(
    df_plot,
    x=y_score,
    color=df_plot.cluster,
    nbins=20,
    facet_col='target',
    facet_row='phase',
    labels=dict(color='True Labels', x='Score'),
    category_orders={'phase': [
        'training',
        "validation",
        "testing"],
        'cluster': clusters}
)
fig.update_layout(
    width=900, height=600,
    showlegend=True,
    title='Target and scores',
    title_x=0.5,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    legend=dict(
        yanchor="top",
        y = 1.05,
        xanchor="left",
        x=0.87
        )
)
fig.update_xaxes(range=[0, 1], constrain='domain')
fig.update_yaxes(range=[0, 750], constrain='domain')
fig.show()
fig.write_html(os.path.join(img_path, 'target_scores.html'))

### AUC and PR curves

In [9]:
fig = make_subplots(rows=1, cols=2, subplot_titles = ['ROC Curves (AUC)', 'PR Curves (AUCPR)'], horizontal_spacing = 0.05)
colors = ["darkcyan", "coral", "cornflowerblue"]

for idx, set in enumerate(['training', 'validation', 'testing']):
    df_plot_phase = df_plot[(df_plot.phase == set)]
    y_true = df_plot_phase.target
    y_score = np.array(df_plot_phase.output.values.tolist())[:, 1]

    fpr_roc, tpr_roc, thr_roc = roc_curve(y_true, y_score)
    pr_pr, rec_pr, thr_pr = precision_recall_curve(y_true, y_score)

    name_roc = f'AUC={auc(fpr_roc, tpr_roc):.4f}'
    name_pr = f'AUCPR={average_precision_score(y_true, y_score):.4f}'

    fig.add_trace(go.Scatter(
        x=fpr_roc,
        y=tpr_roc,
        name=name_roc,
        mode='markers+lines',
        legendgroup=set,
        legendgrouptitle_text=f"{set}",
        marker_color = colors[idx]),
        row=1,
        col=1)
    fig.add_trace(go.Scatter(
        x=rec_pr,
        y=pr_pr,
        name=name_pr,
        mode='markers+lines',
        legendgroup=set,
        marker_color = colors[idx]),
        row=1,
        col=2)

fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1,
    row = 1, col = 1
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0,
    row = 1, col = 2 
)
fig.update_layout(
    width=900, height=400,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    legend=dict(
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=0.95
        ))
fig.update_xaxes(title_text="FPR", constrain='domain', scaleratio = 1, row=1, col=1)
fig.update_yaxes(title_text="TPR (Recall)", constrain='domain', scaleanchor = "x", scaleratio = 1, row=1, col=1)
fig.update_xaxes(title_text="Recall", constrain='domain', scaleanchor = "y", scaleratio = 1, row=1, col=2)
fig.update_yaxes(title_text="Precision", constrain='domain', scaleratio = 1, row=1, col=2)
fig.write_html(os.path.join(img_path, 'auc_aucpr.html'))
fig.show()

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1018: UndefinedMetricWarning:

No positive samples in y_true, true positive value should be meaningless

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:879: UserWarning:

No positive class found in y_true, recall is set to one for all thresholds.

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:879: UserWarning:

No positive class found in y_true, recall is set to one for all thresholds.

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1018: UndefinedMetricWarning:

No positive samples in y_true, true positive value should be meaningless

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:879: UserWarning:

No positive class found in y_true, reca

### Metrics vs thresholds curves

In [10]:
d = {'thr': [], 'precision': [], 'recall': [], 'accuracy': [], 'f1': [], 'mcc': [], 'phase': []}
thr_df = pd.DataFrame(data=d)

for idx, set in enumerate(['training', 'validation', 'testing']):
    df_plot_phase = df_plot[(df_plot.phase == set)]
    y_true = df_plot_phase.target
    y_score = np.array(df_plot_phase.output.values.tolist())[:, 1]

    thrs = np.linspace(0,1,100)
    precision = []
    recall = []
    accuracy = []
    f1 = []
    mcc = []
    
    for thr in thrs:
        y_pred = (y_score > thr)*1
        precision.append(precision_score(y_true, y_pred))
        recall.append(recall_score(y_true, y_pred))
        accuracy.append(accuracy_score(y_true, y_pred))
        f1.append(f1_score(y_true, y_pred))
        mcc.append(matthews_corrcoef(y_true, y_pred))
    phase_df = pd.DataFrame({'thr': thrs, 'precision': precision, 'recall': recall, 'accuracy': accuracy, 'f1': f1, 'mcc': mcc, 'phase': set})
    thr_df = pd.concat([thr_df, phase_df], ignore_index=True)

idx_mcc_max = thr_df.mcc.idxmax()
sel_thr = thr_df.loc[idx_mcc_max].thr

fig_thresh = px.line(
    thr_df,
    x='thr',
    y=[
        'precision',
        'recall',
        'accuracy',
        'f1',
        'mcc'
    ],
    facet_col='phase',
    category_orders={'phase': [
        'training',
        "validation",
        "testing"]},
    width=1000,
    height=500
)
fig_thresh.add_vline(x=sel_thr, line_width=3, line_dash="dash", line_color="green")
fig_thresh.update_layout(
    title='Metrics vs thresholds',
    title_x=0.5)
fig_thresh.update_yaxes(range=[-0.2, 1.2], scaleanchor="x", scaleratio=1, constrain='domain')
fig_thresh.update_xaxes(range=[0, 1], scaleratio = 1, constrain='domain')
fig_thresh.write_html(os.path.join(img_path, 'thresholds_metrics.html'))
fig_thresh.show()


/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning:

Precision is ill-def

### Saving metadata

In [11]:
thr_df.to_hdf(
    os.path.join(metrics_path, f'epoch{epoch}_thresholds.hdf5'),
    key='thresholds',
    mode='w')
    
exp_df.at[exp_id, 'train_loss'] = df[(df.epoch == epoch) & (df.phase == 'training')].loss.mean()
exp_df.at[exp_id, 'valid_loss'] = df[(df.epoch == epoch) & (df.phase == 'validation')].loss.mean()
exp_df.at[exp_id, 'test_loss'] = df[(df.epoch == 0) & (df.phase == 'testing')].loss.mean()
exp_df.at[exp_id, 'train_accuracy'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'training')].accuracy), 3)
exp_df.at[exp_id, 'valid_accuracy'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'validation')].accuracy), 3)
exp_df.at[exp_id, 'test_accuracy'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'testing')].accuracy), 3)
exp_df.at[exp_id, 'train_mcc'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'training')].mcc), 3)
exp_df.at[exp_id, 'valid_mcc'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'validation')].mcc), 3)
exp_df.at[exp_id, 'test_mcc'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'testing')].mcc), 3)
exp_df.at[exp_id, 'train_f1'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'training')].f1), 3)
exp_df.at[exp_id, 'valid_f1'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'validation')].f1), 3)
exp_df.at[exp_id, 'test_f1'] = round(float(thr_df[(thr_df.thr == sel_thr) & (thr_df.phase == 'testing')].f1), 3)

with pd.ExcelWriter(
    'experiments_log.xlsx',
    engine="openpyxl",
    mode="w",
) as writer:
    exp_df.to_excel(writer, sheet_name='All', index=exp_id, header=True)

exp_df

,input_data_path,protein_class,target_data,res_data,task,node_features,edge_features,net,batch_size,optimizer,...,test_accuracy,train_mcc,valid_mcc,test_mcc,train_f1,valid_f1,test_f1,train_rmse,valid_rmse,test_rmse
exp_id,,,,,,,,,,,,,,,,,,,,,
exp3,/Users/giuliacrocioni/Desktop/docs/eScience/pr...,I,BA,residue,classif,"['res_type', 'res_charge', 'res_size', 'polari...","['same_chain', 'distance', 'covalent', 'electr...",<class 'deeprankcore.naive_gnn.NaiveNetwork'>,16,<class 'torch.optim.adam.Adam'>,...,0.0,0.0,0.0,0.0,0.217,0.0,0.0,NaN,NaN,NaN
